Klauss tomo simulation

In [2]:
import numpy as np
import numpy.random as nr
import scipy.stats as ss
import tifffile, os
import multiprocess as mp
from numba import jit, prange
from dask import delayed

In [4]:


"""
    This script is for simulating multi-dimensional data for Klaus. The raw 3D
    is grabbed from https://www.digitalrocksportal.org/projects/6
    The 3D data is cropped from the original data.
"""
######### numba
ph1_cen = 41
ph1_std = 11

ph2_cen = 68
ph2_std = 9.5

ph3_cen = 82
ph3_std = 9.3

ph4_cen = 141
ph4_std = 9.2

y_p1 = ss.norm(ph1_cen, ph1_std).pdf
y_p2 = ss.norm(ph2_cen, ph2_std).pdf
y_p3 = ss.norm(ph3_cen, ph3_std).pdf
y_p4 = ss.norm(ph4_cen, ph4_std).pdf

p = np.ndarray([4])

#@jit
def ratio(gray_val):   
    p[0] = y_p1(gray_val)
    p[1] = y_p2(gray_val)
    p[2] = y_p3(gray_val)
    p[3] = y_p4(gray_val)
    ps = p.sum()
    #return (p[:i].sum()/ps for i in range(4))
    p0 = p[:1].sum()/ps
    p1 = p[:2].sum()/ps
    p2 = p[:3].sum()/ps
    p3 = p[:4].sum()/ps
    return p0, p1, p2, p3

#@jit
def discriminator(gray_val):
    c1 = 0
    c2 = 0 
    c3 = 0
    c4 = 0
    p1, p2, p3, p4 = ratio(gray_val)
    for ii in prange(100):
        seat = nr.random()
        if seat<p1:
            c1 += 1
        elif seat<p2:
            c2 += 1
        elif seat<p3:
            c3 += 1
        else:
            c4 += 1
    return c1, c2, c3, c4


data = tifffile.imread("/media/xiao_usb/originating/256x256x256.tif").reshape([-1])
n_cpu = os.cpu_count()
dim = data.shape
with mp.Pool(n_cpu-1) as pool:
    rlt = pool.starmap(discriminator, [data[ii] for ii in range(int(1e4))])
pool.join()
pool.close()

rlt = []
for ii in prange(dim[0]):
    rlt.append(discriminator(data[ii]))

TypeError: 'numpy.uint8' object is not iterable

In [ ]:
a = [data[ii] for ii in range(int(1e4))]
print(type(a))
dir(tuple.__iter__)

In [ ]:
########## dask
ph1_cen = 41
ph1_std = 11

ph2_cen = 68
ph2_std = 9.5

ph3_cen = 82
ph3_std = 9.3

ph4_cen = 141
ph4_std = 9.2

y_p1 = ss.norm(ph1_cen, ph1_std).pdf
y_p2 = ss.norm(ph2_cen, ph2_std).pdf
y_p3 = ss.norm(ph3_cen, ph3_std).pdf
y_p4 = ss.norm(ph4_cen, ph4_std).pdf

p = np.ndarray([4])

def ratio(gray_val):   
    p[0] = y_p1(gray_val)
    p[1] = y_p2(gray_val)
    p[2] = y_p3(gray_val)
    p[3] = y_p4(gray_val)
    ps = p.sum()
    #return (p[:i].sum()/ps for i in range(4))
    p0 = p[:1].sum()/ps
    p1 = p[:2].sum()/ps
    p2 = p[:3].sum()/ps
    p3 = p[:4].sum()/ps
    return p0, p1, p2, p3

def discriminator(gray_val):
    c1 = 0
    c2 = 0 
    c3 = 0
    c4 = 0
    p1, p2, p3, p4 = ratio(gray_val)
    #for ii in prange(100):
    seat = nr.random()
    if seat<p1:
        c1 += 1
    elif seat<p2:
        c2 += 1
    elif seat<p3:
        c3 += 1
    else:
        c4 += 1
    return c1, c2, c3, c4


data = tifffile.imread("/media/xiao_usb/originating/256x256x256.tif").reshape([-1])
n_cpu = os.cpu_count()
dim = data.shape
print(dim)


results = []
for ii in range(dim[0]):
    rlt = delayed(discriminator)(data[ii])
    results.append(rlt)
dask.compute(*results)

In [1]:
import numpy as np
import numpy.random as nr
import scipy.stats as ss
import tifffile, os, time
import multiprocess as mp

In [61]:
########## multiprocess

"""
    This script is for simulating multi-dimensional data for Klaus. The raw 3D
    is grabbed from https://www.digitalrocksportal.org/projects/6
    The 3D data is cropped from the original data.
"""

ph1_cen = 41
ph1_std = 11

ph2_cen = 68
ph2_std = 9.5

ph3_cen = 82
ph3_std = 9.3

ph4_cen = 141
ph4_std = 9.2

y_p1 = ss.norm(ph1_cen, ph1_std).pdf
y_p2 = ss.norm(ph2_cen, ph2_std).pdf
y_p3 = ss.norm(ph3_cen, ph3_std).pdf
y_p4 = ss.norm(ph4_cen, ph4_std).pdf

p = np.ndarray([4])

def ratio(gray_val):   
    p[0] = y_p1(gray_val)
    p[1] = y_p2(gray_val)
    p[2] = y_p3(gray_val)
    p[3] = y_p4(gray_val)
    ps = p.sum()
    # print(ps)
    return list(p[:i].sum()/ps for i in range(1, 5))
    
def discriminator(gray_val):
    c1 = c2 = c3 = c4 = 0
    p1, p2, p3, p4 = ratio(gray_val)
    for ii in range(1000):
        seat = nr.random()
        if seat<p1:
            c1 += 1
        elif seat<p2:
            c2 += 1
        elif seat<p3:
            c3 += 1
        else:
            c4 += 1
    return c1, c2, c3, c4
    # return nr.random()

# if __name__ == "__main__":
data = tifffile.imread("/media/xiao_usb/originating/256x256x256.tif").reshape([-1])
n_cpu = os.cpu_count()
dim = data.shape
print(data.shape, data[100])
results = []

print(time.asctime())
with mp.Pool(maxtasksperchild=10) as pool:
    rlt = pool.map(discriminator, [data[ii] for ii in np.int32(np.arange(dim[0]))])
    # results.append(rlt)
    # print(rlt)       
pool.close()
pool.join()

print(time.asctime())
print(len(rlt))

(16777216,) 96
Fri Jun 12 23:03:07 2020
Fri Jun 12 23:34:40 2020
16777216


In [47]:

print(ratio(80))
print(y_p1(80))
print(rlt[11910600])

[0.0, 0.001110046443019528, 0.31166018420855524, 0.9999999997976192]
6.759626871616375e-05
(0, 8, 559, 433)


In [62]:
results2 = np.array(rlt).reshape([256, 256, 256, 4])

In [63]:
tifffile.imsave("/media/xiao_usb/originating/channel1.tif", results2[:, :, :, 0].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/channel2.tif", results2[:, :, :, 1].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/channel3.tif", results2[:, :, :, 2].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/channel4.tif", results2[:, :, :, 3].astype(np.float32))

In [57]:
%matplotlib qt
import matplotlib.pyplot as plt
import napari
plt.imshow(results2[:, :, 0, 0])

In [58]:
with napari.gui_qt():
    napari.view_image(results2)

In [ ]:
(16777216,) 96
Fri Jun 12 21:50:17 2020
Fri Jun 12 21:50:24 2020
100000

(16777216,) 96
Fri Jun 12 21:51:12 2020
Fri Jun 12 21:51:18 2020
100000



(16777216,) 96
Fri Jun 12 21:09:54 2020
Fri Jun 12 21:22:50 2020
16777216

In [128]:
a = np.array([2, 1], dtype=np.float32)
b = np.tile(a[:, np.newaxis, np.newaxis], [1, 1000, 1000])
c = np.tile(np.arange(100).astype(np.float32)[:, np.newaxis], [1, 2])

e = (a * c)
e1 = (a * c).sum(axis=1)
#d = (b * c[:, :, np.newaxis, np.newaxis, np.newaxis])
d1 = (b * c[:, :, np.newaxis, np.newaxis]).sum(axis=1)


print(a.shape)
print(c.shape)
print(e.shape)
#print(e[:, :, ...])
print(e1.shape)

print(b.shape)
#print(d.shape)
print(d1.shape)



(2,)
(100, 2)
(100, 2)
(100,)
(2, 1000, 1000)
(100, 1000, 1000)


In [134]:
np.vstack((np.ones([10])*2, np.ones([10]))).T.shape

(10, 2)

In [136]:
d1.sum(axis=0).shape

(1000, 1000)